# <span style="color: green; font-size: 40px; font-weight: bold;"> Projeto (Regressão) </span>

<br> <br>

# Prevendo a Cotação de Criptomoedas em Tempo Real com PySpark e Machine Learning

<br>

### Contexto

Neste mini-projeto, vamos explorar um importante contexto de negócio na área de finanças: a **previsão da cotação de criptomoedas**. O projeto será desenvolvido desde a concepção do problema de negócio até a entrega de um modelo preditivo, utilizando ferramentas comuns de análise de dados no dia a dia de um Cientista de Dados. Apesar do foco ser em ferramentas de análise de dados, o projeto não serve como aconselhamento financeiro.

<br>

### Objetivo

O objetivo deste mini-projeto é **construir um modelo de Machine Learning capaz de prever a cotação de criptomoedas**. Usaremos dados históricos do Bitcoin para treinar o modelo. O Bitcoin, lançado em 2009 pelo anônimo Satoshi Nakamoto, é a criptomoeda mais antiga e conhecida. Servindo como meio descentralizado de troca digital, as transações de Bitcoin são verificadas e registradas em um livro público distribuído chamado Blockchain. O modelo deve ser capaz de prever a cotação do Bitcoin em tempo real a partir de novos dados de entrada. Este projeto pode ser estendido para outras criptomoedas ou instrumentos financeiros com dados de cotação disponíveis.

<br>

### Pergunta de Negócio Principal

> "Como podemos prever a cotação futura do Bitcoin usando dados históricos?"

<br>

### Entregável

O entregável deste mini-projeto será um **modelo de Machine Learning treinado para prever a cotação do Bitcoin**. O modelo será desenvolvido utilizando dados históricos de cotação do Bitcoin e será capaz de fazer previsões em tempo real com base em novos dados de entrada. O processo incluirá a concepção do problema de negócio, preparação dos dados, desenvolvimento do modelo, e a entrega do modelo preditivo.

<br>

### Sobre o Conjunto de Dados

Os dados utilizados neste mini-projeto abrangem o período de 2011 a 2021. O arquivo CSV contém registros OHLC (Open, High, Low, Close) da cotação do Bitcoin, Volume em BTC e Volume na moeda (dólar).

A última coluna indica o preço ponderado do Bitcoin. Os carimbos de data/hora (timestamp) estão em hora Unix. Timestamps sem atividade têm seus campos de dados preenchidos com NaNs. Se estiver faltando um carimbo de data/hora ou houver saltos, isso pode ser devido à inatividade da Exchange, inexistência da Exchange, ou outros erros técnicos na coleta dos dados.

Optamos por não usar dados do ano de 2022 devido à sua natureza atípica, mas você pode incluir esses dados e treinar novamente o modelo se desejar.

<br>

Para este projeto, utilizaremos o conjunto de dados "Bitcoin Historical Data", que contém quase 5 milhões de linhas e informações detalhadas sobre as transações de Bitcoin ao longo do tempo. O conjunto de dados inclui dados sobre preços de abertura, fechamento, máximo e mínimo, volume de Bitcoins negociados, volume em moeda fiduciária e preço ponderado. Além disso, uma coluna adicional dateTime foi criada para converter os timestamps Unix em um formato de data e hora legível.

<br>

<table border="2">
  <tr>
    <th style="text-align: center; font-size: 16px;">Nome da Coluna</th>
    <th style="text-align: center; font-size: 16px;">Tipo de Dado</th>
    <th style="text-align: center; font-size: 16px;">Descrição</th>
  </tr>
  <tr>
    <td>Timestamp</td>
    <td>integer</td>
    <td>Representa o Unix timestamp, que é o número de segundos desde 1 de janeiro de 1970 (UTC).</td>
  </tr>
  <tr>
    <td>Open</td>
    <td>double</td>
    <td>Preço de abertura do Bitcoin no início do período de tempo.</td>
  </tr>
  <tr>
    <td>High</td>
    <td>double</td>
    <td>Preço mais alto do Bitcoin durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Low</td>
    <td>double</td>
    <td>Preço mais baixo do Bitcoin durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Close</td>
    <td>double</td>
    <td>Preço de fechamento do Bitcoin no final do período de tempo.</td>
  </tr>
  <tr>
    <td>Volume_(BTC)</td>
    <td>double</td>
    <td>Volume total de Bitcoins negociados durante o período de tempo.</td>
  </tr>
  <tr>
    <td>Volume_(Currency)</td>
    <td>double</td>
    <td>Volume total em moeda fiduciária (por exemplo, USD) das transações de Bitcoin durante o período.</td>
  </tr>
  <tr>
    <td>Weighted_Price</td>
    <td>double</td>
    <td>Preço ponderado do Bitcoin, calculado com base nos preços e volumes das transações durante o período.</td>
  </tr>
  <tr>
    <td>dateTime</td>
    <td>string</td>
    <td>Irá representar a data e hora no formato legível, será criada a partir do Unix timestamp. <b>(Nova Coluna)</b> </td>
  </tr>
</table>


<br> <br> <br>

# Importando Pacotes

In [1]:
### Imports

# Importa o findspark e inicializa
import findspark
findspark.init()

## Bibliotecas de Manipulação e Análise de Dados

import pandas as pd                      # Biblioteca para manipulação e análise de dados tabulares.
import numpy as np                       # Biblioteca para cálculos numéricos e manipulação de arrays.


## Bibliotecas de Visualização de Dados

import seaborn as sns                    # Biblioteca para visualização de dados estatísticos.
from matplotlib import pyplot as plt     # Biblioteca para criação de gráficos e visualizações.


## Bibliotecas Principais do PySpark

import pyspark                           # Biblioteca para processamento de dados em grande escala usando clusters.
from pyspark import SparkConf            # Configuração e criação do contexto do Spark.
from pyspark import SparkContext         # Configuração e criação do contexto do Spark.   
from pyspark.sql import SparkSession     # Criação e manipulação de sessões e contextos SQL no Spark.
from pyspark.sql import SQLContext       # Criação e manipulação de sessões e contextos SQL no Spark.
from pyspark.sql.types import *          # Tipos de dados usados na criação de schemas de DataFrames no Spark.
from pyspark.sql.functions import *      # Funções SQL usadas para manipulação e transformação de dados no Spark.
from pyspark.sql.functions import col, count, when, isnan

## Bibliotecas de Machine Learning no PySpark

from pyspark.ml.linalg import Vectors         # Estruturas de dados para manipulação de vetores na MLlib do Spark.
from pyspark.ml.feature import StringIndexer  # Transformação de variáveis categóricas em numéricas.
from pyspark.ml.regression import LinearRegression      # Algoritmo de regressão linear na MLlib do Spark.
from pyspark.mllib.evaluation import RegressionMetrics  # Métricas de avaliação para modelos de regressão.
from pyspark.ml.stat import Correlation                 # Cálculo de correlações entre colunas de DataFrames.
from pyspark.ml.feature import MinMaxScaler             # Normalização dos dados para um intervalo específico.
from pyspark.ml.feature import VectorAssembler   # Combinação de múltiplas colunas em uma única coluna de vetores.
from pyspark.ml import Pipeline          # Construção de pipelines de ML que consistem em uma sequência de etapas.
from pyspark.ml.tuning import ParamGridBuilder    # Ferramentas para construção de grids de parâmetros.
from pyspark.ml.tuning import CrossValidator      # Ferramentas para construção de validação cruzada.
from pyspark.ml.tuning import CrossValidatorModel # Ferramentas para construção de validação cruzada.
from pyspark.ml.feature import StandardScaler     # Normalização de dados para ter média zero e variância unitária.
from pyspark.ml.evaluation import RegressionEvaluator  # Avaliação de modelos de regressão utilizando 
                                                       # métricas como RMSE e R2.

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [2]:
## Formatação das saídas

# Configura o Pandas para mostrar até 200 colunas ao exibir um DataFrame
pd.set_option('display.max_columns', 200)

# Configura o Pandas para mostrar até 400 caracteres por coluna ao exibir um DataFrame
pd.set_option('display.max_colwidth', 400)

# Configuramos matplotlib_axes_logger para exibir apenas mensagens de erro 
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

findspark : 2.0.1
numpy     : 1.22.4
sys       : 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
matplotlib: 3.4.3
decimal   : 1.70
pyspark   : 3.5.1
seaborn   : 0.11.2
pandas    : 1.3.5



<br>

## Por que vamos usar o <i>PySpark</i> ao invés de utilizarmos somente <i>Linguagem Python</i>?

> Antes de responder, podemos afirmar que SIM, este projeto poderia ser feito usando somente <i>Linguagem Python</i>.

#### Então por que usar o PyStark?

Como iremos ver a seguir, nosso conjunto de dados possui um tamanho de **317MB** e **quase 5 milhões de linhas**. Será que conseguiríamos processar esse volume de dados tão alto com Linguagem Python? Provavelmente não!

<br>

Portanto usaremos o **PySpark** pois o ele nos permite trabalhar em um ambiente distribuído. 

<br>

# Preparando o Ambiente Spark

In [4]:
# Definindo semente aleatória (seed) para reprodutibilidade do notebook
rnd_seed = 23
np.random.seed = rnd_seed
np.random.set_state = rnd_seed

# Criando o Spark Context
conf = SparkConf().setAppName("Mini-Projeto3") \
                  .set("spark.ui.showConsoleProgress", "false") \
                  .set("spark.executor.heartbeatInterval", "20s") \
                  .set("spark.eventLog.enabled", "false") \
                  .set("spark.sql.shuffle.partitions", "2") \
                  .set("spark.sql.debug.maxToStringFields", "100")

# Criar o Spark Context e a Spark Session
sc = SparkContext(conf=conf)
spark_session = SparkSession.builder.config(conf=conf).getOrCreate()

# Ajustar o nível de log para ERROR
sc.setLogLevel("ERROR")

# Configurar log4j para suprimir avisos (deixar como comentário e volta ao normal)
log4j_logger = sc._jvm.org.apache.log4j
log4j_logger.LogManager.getLogger("org").setLevel(log4j_logger.Level.ERROR)
log4j_logger.LogManager.getLogger("akka").setLevel(log4j_logger.Level.ERROR)

# Visualizar o objeto spark_session
spark_session

24/07/11 18:37:12 WARN Utils: Your hostname, eduardo-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface wlp0s20f3)
24/07/11 18:37:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/11 18:37:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<br>

# Carregando os Dados

In [5]:
# Carrega os dados a partir da sessão Spark
df_spark = spark_session.read.csv('dados/dataset.csv', header = 'true', inferSchema = 'true')

In [6]:
# Tipo do objeto
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [7]:
# Visualiza as 5 primeiras linhas dos dados
df_spark.show(5)

+----------+----+----+----+-----+------------+-----------------+--------------+
| Timestamp|Open|High| Low|Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+----------+----+----+----+-----+------------+-----------------+--------------+
|1325317920|4.39|4.39|4.39| 4.39|  0.45558087|     2.0000000193|          4.39|
|1325319300| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319360| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319420| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
|1325319480| NaN| NaN| NaN|  NaN|         NaN|              NaN|           NaN|
+----------+----+----+----+-----+------------+-----------------+--------------+
only showing top 5 rows



In [8]:
# Visualiza os metadados (schema) - similar ao info()
df_spark.printSchema()

root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



In [9]:
# Converte o DataFrame do Spark para um DataFrame do Pandas (apenas se quiser)
#df_pandas = df_spark.toPandas()

# Visualiza os dados usando display()
#display(df_pandas.head(5))

<br><br><br>

# <span style="color: green; font-size: 34px; font-weight: bold;"> Análise Exploratória de Dados </span>

<br>

### Análise Inicial

In [24]:
def analise_inicial(df):
    # Exibir o schema do DataFrame
    print('\n\n INFO \n\n')
    df.printSchema()
    print('\n\n ------------------------------------------------------------------------------------------ \n\n')

    # Verificar valores ausentes em cada coluna
    print('\n TOTAL DE VALORES NaN por Coluna \n')
    valores_ausentes = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
    valores_ausentes.show()

    # Verificar se existe alguma variável com valores ausentes
    valores_ausentes_boolean = any(row[c] > 0 for row in valores_ausentes.collect() for c in df.columns)

    # Nomes das variáveis com valores ausentes
    variaveis_ausentes = [c for c in df.columns if df.filter(col(c).isNull() | isnan(c)).count() > 0]

    # Número de linhas duplicadas
    num_linhas_total = df.count()
    num_linhas_distintas = df.distinct().count()
    num_linhas_duplicadas = num_linhas_total - num_linhas_distintas

    # Porcentagem de linhas duplicadas
    porcentagem_linhas_duplicadas = (num_linhas_duplicadas / num_linhas_total) * 100

    # Total de linhas com pelo menos um valor ausente
    total_linhas_ausentes = df.filter(
        " OR ".join([f"(`{c}` IS NULL OR isnan(`{c}`))" for c in df.columns])
    ).count()

    # Porcentagem de linhas com pelo menos um valor ausente
    porcentagem_linhas_ausentes = (total_linhas_ausentes / num_linhas_total) * 100

    # Exibe o resultado
    print("\n\nExistem valores ausentes:", valores_ausentes_boolean)
    if valores_ausentes_boolean:
        print("\nVariáveis com valores ausentes:", variaveis_ausentes)
        print("\nTotal de Linhas com Valores Ausentes:", total_linhas_ausentes)
        print("\nPorcentagem de Linhas Com Valor Ausente: {:.2f}%".format(porcentagem_linhas_ausentes))
    else:
        print("\nNenhuma variável possui valores ausentes.")

    print("\n\n\nExistem valores duplicados:", num_linhas_duplicadas > 0)
    if num_linhas_duplicadas > 0:
        print("\nNúmero de Linhas Duplicadas:", num_linhas_duplicadas)
        print("\nPorcentagem de Linhas Duplicadas: {:.2f}%\n".format(porcentagem_linhas_duplicadas))
    else:
        print("\nNenhuma variável possui valores duplicados.\n")

# Exemplo de uso da função com o DataFrame df_spark
analise_inicial(df_spark)



 INFO 


root
 |-- Timestamp: integer (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume_(BTC): double (nullable = true)
 |-- Volume_(Currency): double (nullable = true)
 |-- Weighted_Price: double (nullable = true)



 ------------------------------------------------------------------------------------------ 



 TOTAL DE VALORES NaN por Coluna 

+---------+-------+-------+-------+-------+------------+-----------------+--------------+
|Timestamp|   Open|   High|    Low|  Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+---------+-------+-------+-------+-------+------------+-----------------+--------------+
|        0|1242831|1242831|1242831|1242831|     1242831|          1242831|       1242831|
+---------+-------+-------+-------+-------+------------+-----------------+--------------+



Existem valores ausentes: True

Variáveis com valores ausentes: ['Open',

<br><br>

# Analisando os Dados

<br>

#### Visualizando Variáveis Categóricas e Numéricas

- Todas as variáveis são numéricas neste dataframe

<br><br>

## Analisando Todas as Variáveis

#### Resumo Estatístico

In [26]:
# Visualizar o resumo estatístico do DataFrame
df_spark.describe().show()

+-------+-------------------+-------+-------+-------+-------+------------+-----------------+--------------+
|summary|          Timestamp|   Open|   High|    Low|  Close|Volume_(BTC)|Volume_(Currency)|Weighted_Price|
+-------+-------------------+-------+-------+-------+-------+------------+-----------------+--------------+
|  count|            4856600|4856600|4856600|4856600|4856600|     4856600|          4856600|       4856600|
|   mean|1.471324115749862E9|    NaN|    NaN|    NaN|    NaN|         NaN|              NaN|           NaN|
| stddev|8.426671569710898E7|    NaN|    NaN|    NaN|    NaN|         NaN|              NaN|           NaN|
|    min|         1325317920|    3.8|    3.8|    1.5|    1.5|         0.0|              0.0|           3.8|
|    max|         1617148800|    NaN|    NaN|    NaN|    NaN|         NaN|              NaN|           NaN|
+-------+-------------------+-------+-------+-------+-------+------------+-----------------+--------------+



<br>

# Limpeza nos Dados

<br>

### Tratando Valores Ausentes

<br><br>

# Features Engineering (se necessário)

<br>

#### Criando nova variável datetime

<br><br>

# Análise Exploratória

<br><br>

## Verificando Correlação

### Conclusão

- Combinando a **análise de correlação** e a **análise dos gráficos**, a recomendação é

<br><br><br><br>

# Pré-Processamento de Dados Para Construção de Modelos de Machine Learning

<br><br>


## Dividindo os dados em Dados de Treino e Dados de Teste
- Nós **treinamos** o modelo com **dados de treino** e **avaliamos** o modelo com **dados de teste**.

<br>